In [14]:
#!/usr/bin/env python3
"""
FPL Draft prep: export top players by position to CSV.

Ranking (simple, position-aware, descending):
- FORWARDS: total_points, points_per_game, minutes, ict_index, goals_scored, assists
- MIDFIELD: total_points, points_per_game, minutes, ict_index, assists, goals_scored
- DEFENDERS: total_points, points_per_game, minutes, ict_index, clean_sheets, goals_scored
- GOALKEEPERS: total_points, points_per_game, minutes, ict_index, clean_sheets, saves

You can tweak the sort keys below or flip include_unavailable=True to keep injured/suspended.
"""
import pathlib
import requests
import pandas as pd

BOOTSTRAP = "https://fantasy.premierleague.com/api/bootstrap-static/"
OUTDIR = pathlib.Path("fpl_shortlist")
OUTDIR.mkdir(exist_ok=True)

# ---- config ----
TOPS = {"FWD": 15, "MID": 25, "DEF": 25, "GK": 10}
include_unavailable = False  # set True to include injured/suspended/etc.
# ----------------

POS_MAP = {1: "GK", 2: "DEF", 3: "MID", 4: "FWD"}

def fetch_bootstrap():
    r = requests.get(BOOTSTRAP, timeout=30)
    r.raise_for_status()
    return r.json()

def tidy_players(data):
    el = pd.DataFrame(data["elements"]).copy()

    # Map element_type -> position; team id -> name
    teams = pd.DataFrame(data["teams"])[["id", "name"]].rename(columns={"id":"team","name":"team_name"})
    types = pd.DataFrame(data["element_types"])[["id","singular_name_short"]].rename(columns={"id":"element_type","singular_name_short":"pos_name"})

    el = el.merge(teams, on="team", how="left")
    el = el.merge(types, on="element_type", how="left")
    el["pos"] = el["element_type"].map(POS_MAP)

    # Keep handy columns
    keep = [
        "id","first_name","second_name","web_name","pos","team_name","status",
        "now_cost","total_points","points_per_game","minutes",
        "goals_scored","assists","clean_sheets","saves",
        "influence","creativity","threat","ict_index",
        "chance_of_playing_next_round","chance_of_playing_this_round",
        "selected_by_percent"
    ]
    el = el[keep]

    # Cast numerics
    to_num = ["now_cost","total_points","points_per_game","minutes","goals_scored",
              "assists","clean_sheets","saves","influence","creativity","threat","ict_index",
              "chance_of_playing_next_round","chance_of_playing_this_round","selected_by_percent"]
    for c in to_num:
        el[c] = pd.to_numeric(el[c], errors="coerce")

    # Availability filter
    if not include_unavailable:
        el = el[el["status"].eq("a")].copy()

    # Friendly display columns
    el["player"] = el["web_name"]
    el["name"] = (el["first_name"].str.strip() + " " + el["second_name"].str.strip()).str.strip()
    return el

def rank_by_position(df, pos):
    dfp = df[df["pos"].eq(pos)].copy()

    if pos == "FWD":
        sort_keys = ["total_points","points_per_game","minutes","ict_index","goals_scored","assists"]
    elif pos == "MID":
        sort_keys = ["total_points","points_per_game","minutes","ict_index","assists","goals_scored"]
    elif pos == "DEF":
        sort_keys = ["total_points","points_per_game","minutes","ict_index","clean_sheets","goals_scored"]
    elif pos == "GK":
        sort_keys = ["total_points","points_per_game","minutes","ict_index","clean_sheets","saves"]
    else:
        raise ValueError("Unknown position")

    dfp = dfp.sort_values(sort_keys, ascending=[False]*len(sort_keys))
    cols_out = [
        "pos","player","name","team_name",
        "total_points","points_per_game","minutes",
        "goals_scored","assists","clean_sheets","saves",
        "ict_index","now_cost","status","selected_by_percent"
    ]
    return dfp[cols_out]



In [15]:

import numpy as np

# Fetch + tidy
data = fetch_bootstrap()
players = tidy_players(data)

# Per-position ranked CSVs
pieces = []
for pos, n in TOPS.items():
    ranked = rank_by_position(players, pos).head(n).copy()
    ranked.insert(0, "rank", range(1, len(ranked) + 1))
    out_path = OUTDIR / f"{pos.lower()}_top{n}.csv"
    ranked.to_csv(out_path, index=False)
    pieces.append(ranked)

# Combined shortlist (tops from each position)
combined = pd.concat(pieces, ignore_index=True)
combined["group_rank"] = (
    combined.groupby("pos")["rank"].rank(method="first").astype(int)
)
combined = combined[
    [
        "pos",
        "rank",
        "player",
        "name",
        "team_name",
        "total_points",
        "points_per_game",
        "minutes",
        "goals_scored",
        "assists",
        "clean_sheets",
        "saves",
        "ict_index",
        "now_cost",
        "selected_by_percent",
    ]
]
(OUTDIR / "ranked_shortlist.csv").write_text(combined.to_csv(index=False))

# Overall priority (no manual overrides)
pri = combined.copy()

def z(col):
    x = pd.to_numeric(pri[col], errors="coerce")
    mu, sd = x.mean(skipna=True), x.std(skipna=True)
    if pd.isna(sd) or sd == 0:
        return pd.Series(0.0, index=pri.index)
    return (x - mu) / sd

z_tp = z("total_points")
z_ppg = z("points_per_game")
z_min = z("minutes")
z_ict = z("ict_index")
z_g = z("goals_scored")
z_a = z("assists")
z_cs = z("clean_sheets")
z_sav = z("saves")

pri["overall_score"] = (
    1.00 * z_tp
    + 0.80 * z_ppg
    + 0.30 * z_min
    + 0.50 * z_ict
    + 0.60 * z_g
    + 0.40 * z_a
    + 0.30 * z_cs
    + 0.10 * z_sav
).fillna(0.0)

pri = pri.sort_values(
    ["overall_score", "total_points", "points_per_game", "name"],
    ascending=[False, False, False, True],
    kind="mergesort",  # stable
)
pri.insert(0, "overall_pick", range(1, len(pri) + 1))

pri_cols = [
    "overall_pick",
    "player",
    "name",
    "pos",
    "team_name",
    "total_points",
    "points_per_game",
    "minutes",
    "goals_scored",
    "assists",
    "clean_sheets",
    "saves",
    "ict_index",
    "now_cost",
    "selected_by_percent",
    "overall_score",
]
out_overall = OUTDIR / "overall_priority.csv"
pri[pri_cols].to_csv(out_overall, index=False)


In [16]:

# New players only (0 or NaN minutes)
mins = pd.to_numeric(pri["minutes"], errors="coerce").fillna(0)
new_players = pri[mins.eq(0)].copy()
new_players = new_players.sort_values(
    ["overall_score", "total_points", "points_per_game", "name"],
    ascending=[False, False, False, True],
    kind="mergesort",
).reset_index(drop=True)
# new_players.insert(0, "overall_pick", range(1, len(new_players) + 1))

new_players_cols = [
    # "overall_pick",
    "player",
    "name",
    "pos",
    "team_name",
    "total_points",
    "points_per_game",
    "minutes",
    "goals_scored",
    "assists",
    "clean_sheets",
    "saves",
    "ict_index",
    "now_cost",
    "selected_by_percent",
    "overall_score",
]
out_new = OUTDIR / "overall_priority_new_players.csv"
new_players[new_players_cols].to_csv(out_new, index=False)

# Console summary
print("Wrote:")
for p in sorted(OUTDIR.glob("*.csv")):
    print(" -", p)

Wrote:
 - fpl_shortlist/def_top25.csv
 - fpl_shortlist/fwd_top15.csv
 - fpl_shortlist/gk_top10.csv
 - fpl_shortlist/mid_top25.csv
 - fpl_shortlist/overall_priority.csv
 - fpl_shortlist/overall_priority_new_players.csv
 - fpl_shortlist/ranked_shortlist.csv
